In [21]:
import requests
import json
from tkinter import *
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
#key = "7b23a8ff2c964c3ca21aa63bb990ab1a"
import html5lib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
from tkinter import messagebox
import webbrowser

options = Options()
options.add_argument('--headless=new')

root = tk.Tk()

frame = ttk.Frame(root)
frame.grid(row=0, column=0, rowspan=4,columnspan=4, sticky="nsew")

canvas = tk.Canvas(frame)
scrollbar = ttk.Scrollbar(frame, orient="vertical", command=canvas.yview)
canvas.configure(yscrollcommand=scrollbar.set)

content_frame = ttk.Frame(canvas) #https://www.tutorialspoint.com/implementing-a-scrollbar-using-grid-manager-on-a-tkinter-window

content_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

canvas.create_window((0, 0), window=content_frame, anchor="nw")
canvas.grid(row=0, column=0, sticky="nsew")
scrollbar.grid(row=0, column=1, sticky="ns")

root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)
frame.columnconfigure(0, weight=1)
frame.rowconfigure(0, weight=1)

#https://stackoverflow.com/questions/23482748/how-to-create-a-hyperlink-with-a-label-in-tkinter

def callback(url):
    webbrowser.open_new(url)




def _on_mousewheel(event):
   canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

canvas.bind_all("<MouseWheel>", _on_mousewheel)

root.geometry("1090x850")
root.title("Recipe Central")

recipe_have_foods = []

#recipe_need_foods = [{"romaine lettuce": "3 leaves"},{"snow peas":350}]

conversion_factors = []


wide_search_input = []


Single_Nutrition_Input = []
wide_search_index = 0



#Lists to later Sum up For Recipe Nutrition Total
calorie_t = []
protein_t = []
fat_t = []
Carbs_t = []
Fiber_t = []
Sugar_t = []
Calci_t = []

clicked = StringVar() 

def Recipe_Maker(recipe_have_foods): #appends a dictonary to recipe_have_foods
    
    
    fname = F_Entry.get()
    gnum = G_Entry.get()
    
    


    try:
        gnum=int(gnum)
        Get_Nutrients_Text.configure(state="normal")
        recipe_have_foods.append({fname:gnum})
        Get_Nutrients_Text.insert(END,fname)
        Get_Nutrients_Text.insert(END,", ")
        Get_Nutrients_Text.configure(state="disabled")
        
    except ValueError:
        tk.messagebox.showinfo(title="Value Error", message="Enter a whole number with no extra characters")
        print(gnum)

    else:
        print("")


    F_Entry.delete(0, END) 
    G_Entry.delete(0, END)
    FG_Button.configure(text="Add Another Food")

def run_conversion_and_wide(recipe_have_foods,wide_search_input, wide_search_index):
    conversion(recipe_have_foods)
    wide_search(wide_search_input,wide_search_index)

    Recipe_Maker_Frame.grid_forget()
    Ingridient_Nutrition_Frame.grid(row=1,column=2,pady=0,padx=0,sticky="ns")
    

    
    
def conversion(recipe_have_foods):
    
# For each recipe ingridient list, I split them apart and appended them to two seperate lists: 
    #Conversion Factors list: Did math to morph the values of each ingridient  + accounted for the fact the some will not be in grams
    #Wide_Search_list: This will be used to do a basic search of all the ingridient ID's in the function below
    for pair in recipe_have_foods:
        for key, value in pair.items():
            if isinstance(value, int) == True:
                wide_search_input.append(key)
                pair[key] = pair[key]/100
                conversion_factors.append(pair[key])

                
                
            else:
                wide_search_input.append(key)
                pair[key] = .1
                conversion_factors.append(pair[key])

    return (conversion_factors, wide_search_input)
    

def wide_search(wide_search_input, wide_search_index):
    import requests
    import json
    
    #I have to run the conversion first or else there would be no wide_search_input to input for choices the user will select (GUI Dropdown)
    
    #running each item in wide Search input (printing the name of ingridient for referance)

    selected_food = StringVar()
    
    #If The User is out of foods, the recipe Total Frame will pop up
    try:
        #the title of the ingridient, will turn into the user selected choice in the root window
        selected_food.set(f"{wide_search_input[wide_search_index]}")
    except IndexError:
        Recipe_Total_Frame.grid(row=2,column=2,pady=0,sticky="ewns")
        recipe_total(calorie_t,protein_t,
        fat_t,
        Carbs_t,
        Fiber_t,
        Sugar_t,
        Calci_t)


    
    try:
        USDA_response = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key=gr1nBpXubK3gdJpQF4vbdcLPppTXREBseSllVNlV&query={wide_search_input[wide_search_index]}&dataType=SR Legacy")

        if USDA_response.status_code == 200:
            USDA_json = USDA_response.json()

            #selection gives the user the choice options on the screen for each ingridient
                #it is currently run in thi8s for loop for that reason ^
            selection(USDA_json, selected_food, wide_search_index)

        else:
            print("Error:", USDA_response.status_code)
            return None
    except IndexError:
        print("all done")
        
 
        
        
def selection(USDA_json, selected_food,wide_search_index):
    food_choices = USDA_json["foods"]
    
    choice_list=[]
    ID_list = []
    for i in food_choices[:10]:
        choice_list.append(i["description"])
        ID_list.append(i["fdcId"])

            
      
    #the dict that I will unpack to get the ID
    food_and_id = dict(zip(choice_list, ID_list))
    #print(food_and_id)


    def Choice_Swapper(wide_search_input,wide_search_index):
        #getting the food name from the selected option
        option = selected_food.get()
        option_label.configure(text=option)
        
        #the ID is its value in the dict
        ID = food_and_id[option]
        ID = int(ID)
        Single_Nutrition_Input.append(ID)
        food_search_narrow(Single_Nutrition_Input)
        wide_search_index += 1
    
        dropdown.destroy()
        choose_button.destroy()
   
        wide_search(wide_search_input,wide_search_index)
        
    
  
        
    
    #creating and packing the dropdown menu based on the current choice list of ingredients 
    
    dropdown = OptionMenu(Ingridient_Nutrition_Frame,selected_food, *choice_list)
    dropdown.pack(side="bottom")
    
    #button for food selection dropdown
    choose_button = Button(Ingridient_Nutrition_Frame, borderwidth=5, highlightbackground="black",text="Select Food", command=lambda: Choice_Swapper(wide_search_input,wide_search_index))
    choose_button.pack(pady=10,side="bottom")
    
    





#This function individually uses (iterates) the IDs selected earlier (in the call itself).
#At the same time it pulls the nutrition information out.

def food_search_narrow(Single_Nutrition_Input):


    import requests
    import json
    #food_id_choice is in the call, its doing each ID at a time

    for f_ID in Single_Nutrition_Input:
        USDA_response = requests.get(f"https://api.nal.usda.gov/fdc/v1/food/{f_ID}?api_key=gr1nBpXubK3gdJpQF4vbdcLPppTXREBseSllVNlV")
        if USDA_response.status_code == 200:
            USDA_json = USDA_response.json()
            #gives the title of the new food (GUI LABEL)
            title = USDA_json["description"].upper()

            
            #runs nutrition for every ID requested
            nutrition(USDA_json, conversion_factors)
            Single_Nutrition_Input.pop(0)
            
            
        
        else:
            print("Error:", USDA_response.status_code)
            return None
        
   
    
                        #uses the converson factors set from earlier
def nutrition(USDA_json,conversion_factors):
    if nut_text == None:
        pass
    else:
        nut_text.delete("1.0", "end")
    
    if not conversion_factors:
        print("Conversion factors list is empty. Cannot calculate nutrition.")
        return

    calorie_u = None
    calorie_v = None
    protein_u = None
    protein_v = None
    fat_u = None
    fat_v = None
    carb_u = None
    carb_v = None
    fib_u = None
    fib_v = None
    sug_u = None
    sug_v = None
    calci_u = None
    calci_v = None 

    #for each instance that there is a specifc nutrient, I print of the nutrient unit +
    #I print off the value taking into account the conversion factor set earlier 
        #It takes the first value from conversion factor to do this, then deletes it at the bottom for the next ingridient to iterate through
    for nutrient in USDA_json["foodNutrients"]:
        if nutrient["nutrient"]["name"] == "Energy":
            calorie_u = "kCal"
            calorie_v = round(nutrient["amount"] * 0.239006 * conversion_factors[0], 2) #kilojul - kcalorie conversion * conversion factor
  
    #Printing off each nutrient and adding the value to the dictionary for later
    nut_text.configure(state="normal")
    nut_text.insert(END,f"Calories:{calorie_v}{calorie_u}\n")
    calorie_t.append(calorie_v)
    

    
    for nutrient in USDA_json["foodNutrients"]:
        if nutrient["nutrient"]["name"] == "Protein":
            protein_u = nutrient["nutrient"]["unitName"]
            protein_v = round(nutrient["amount"] * conversion_factors[0], 2)
  
    nut_text.insert(END,f"Protein: {protein_v}{protein_u}\n")
    protein_t.append(protein_v)
    
    
    for nutrient in USDA_json["foodNutrients"]:
        if nutrient["nutrient"]["name"] == "Total lipid (fat)":
            fat_u = nutrient["nutrient"]["unitName"]
            fat_v = round(nutrient["amount"] * conversion_factors[0],2)
  
    nut_text.insert(END,f"Fat: {fat_v}{fat_u}\n")
    fat_t.append(fat_v)
    
       
    for nutrient in USDA_json["foodNutrients"]:
        if nutrient["nutrient"]["name"] == "Carbohydrate, by difference":
            carb_u = nutrient["nutrient"]["unitName"]
            carb_v = round(nutrient["amount"] * conversion_factors[0],2)
      
    nut_text.insert(END,f"Carbs: {carb_v}{carb_u}\n")
    Carbs_t.append(carb_v)
    
      
    try:
        for nutrient in USDA_json["foodNutrients"]:
            if nutrient["nutrient"]["name"] == "Fiber, total dietary":
                fib_u = nutrient["nutrient"]["unitName"]
                fib_v = round(nutrient["amount"] * conversion_factors[0], 2)
    except NameError:
        fib_v = 0 
        nut_text.insert(END,f"Fiber: {fib_v}{fib_u}\n")

    if fib_v is not None:
        nut_text.insert(END,f"Fiber: {fib_v}{fib_u}\n")
        Fiber_t.append(fib_v)
    else:
        fib_v = 0
    
      
    for nutrient in USDA_json["foodNutrients"]:
        if nutrient["nutrient"]["name"] == "Calcium, Ca":
            calci_u = nutrient["nutrient"]["unitName"]
            calci_v = round(nutrient["amount"] * conversion_factors[0],2)
  
    nut_text.insert(END,f"Calcium: {calci_v}{calci_u}\n")
    Calci_t.append(calci_v)
    
    try:
        for nutrient in USDA_json["foodNutrients"]:
            if nutrient["nutrient"]["name"] == "Total Sugars":
                sug_u = nutrient["nutrient"]["unitName"]
                sug_v = round(nutrient["amount"] * conversion_factors[0], 2)
    except NameError:
        sug_v = 0 
        nut_text.insert(END,f"Sugars: {sug_v}{sug_u}\n")

    if sug_v is not None:
        nut_text.insert(END,f"Sugars: {sug_v}{sug_u}\n")
        Sugar_t.append(sug_v)
    else:
        sug_v = 0
        
    conversion_factors.pop(0)
    nut_text.configure(state="disabled")
    
    
    
    #sums up the values in the dictionary from earlier
def recipe_total(calorie_t,protein_t,
fat_t,
Carbs_t,
Fiber_t,
Sugar_t,                 
Calci_t):
    
    recipeT.configure(state="normal")
    recipeT.delete("1.0", "end")
        
    recipeT.insert(END,f"Calories: {round(sum(calorie_t),2)}kCal\n")
    recipeT.insert(END,f"Protein: {round(sum(protein_t),2)}g\n")
    recipeT.insert(END,f"Fat: {round(sum(fat_t),2)}g\n")
    recipeT.insert(END,f"Carbs: {round(sum(Carbs_t),2)}g\n")
    recipeT.insert(END,f"Fiber: {round(sum(Fiber_t),2)}g\n")
    recipeT.insert(END,f"Calcium: {round(sum(Calci_t),2)}mg\n")
    recipeT.insert(END,f"Sugars: {round(sum(Sugar_t),2)}g\n")
    recipeT.configure(state="disabled")



# donnys gui
api_key = "7b23a8ff2c964c3ca21aa63bb990ab1a"
missed_dict = {}
nutrientDict = {}
def display_recipes():
    food_list = entry.get().split(",")
    recipe_urls = fetch_recipe_data(food_list)
    recipe_list = scrape_recipe_details(recipe_urls)
    
    #Tags for colored Text: source- https://stackoverflow.com/questions/47591967/changing-the-colour-of-text-automatically-inserted-into-tkinter-widget
    output_text.tag_configure("money", foreground="green4")
    output_text.tag_configure("missing", foreground="darkslategray")
    output_text.tag_configure("Title", foreground="blue")
    output_text.delete(1.0, tk.END)

    for recipe in recipe_list:
        output_text.insert(tk.END, f"{recipe['name']}:\n","Title")
        for ingredient, measure in recipe['ingredients'].items():
            output_text.insert(tk.END, f"- {ingredient}: {measure}\n")
        output_text.insert(tk.END,f"Ingredients Missing : {missed_dict[recipe['name']]}\n","missing")
        
        
        output_text.insert(tk.END, f"{recipe['price']}","money")
        
        
        output_text.insert(tk.END, '\n --------------------------\n')
    more_recipe()

invalidfood_label = None
More_Spoonacular_Label = None

def fetch_recipe_data(food_list):
    global invalidfood_label, More_Spoonacular_Label
    sep = ',+'
    food = sep.join(food_list)
    url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={food}&number=5&apiKey={api_key}"
    response = requests.get(url)
    json_file = response.json()
    url_list = []
    global missed_dict
    missed_dict = {}
    if json_file != []:
        url_list = []
    else:
        if invalidfood_label is not None and invalidfood_label.winfo_exists(): 
            invalidfood_label.destroy()
        invalidfood_label = Label(Recipe_Frame, text="Please Enter a Valid Ingredient", font=("Arial", 18), background="darkseagreen")
        invalidfood_label.pack()

        # Destroy the "Enter The Recipe You want Nutrition info for" label if it exists
        if More_Spoonacular_Label is not None and More_Spoonacular_Label.winfo_exists():
            More_Spoonacular_Label.destroy()
        
        return []
    
    # If valid ingredients were entered, pack the "Enter The Recipe You want Nutrition info for" label
    if More_Spoonacular_Label is not None and not More_Spoonacular_Label.winfo_exists():
        More_Spoonacular_Label.pack(padx=(50, 50), pady=10)
    
    for item in json_file:
        id = item["id"]
        price_url = f"https://api.spoonacular.com/recipes/{id}/information?apiKey={api_key}"
        price_response = requests.get(price_url)
        price_data = price_response.json()
        url_list.append(price_data["spoonacularSourceUrl"])
    
    for e in json_file:
        title = e["title"]
        missed_ingredients = []
        for i in e["missedIngredients"]:
            missed_ingredients.append(i["name"])
            missed_dict_per = ({title : missed_ingredients})
            missed_dict.update(missed_dict_per)

    return url_list
  

def scrape_recipe_details(recipe_urls):
    recipe_list = []
    global nutrientDict
    nutrientDict = {}
    global choices
    choices = []



    for url in recipe_urls:
        measures = []
        ingredients = []
        price = []
        recipe_name = ""
        nutrientName = []
        nutrientValue =[]
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        x = 1

        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "spoonacular-metric"))
        )

        input_element = driver.find_element(By.XPATH, "//*[@id='spoonacularMeasure']/label[1]")
        input_element.click()

        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.stepper-wrap"))
        )

        input_element = driver.find_element(By.XPATH, "//*[@id='spoonacular-serving-stepper']")
        input_element.clear()
        input_element.send_keys("0" + Keys.ENTER)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.spoonacular-ingredient")))
        for element in driver.find_elements(By.CSS_SELECTOR, "div.spoonacular-ingredient"):
            measure = element.find_element(By.CSS_SELECTOR, "div.spoonacular-amount.t12.spoonacular-metric").text
            measures.append(measure)
        for element in driver.find_elements(By.CSS_SELECTOR, "div.spoonacular-image-wrapper"):
            while x!= "Done":
                try:
                    ingredient = element.find_element(By.XPATH, f'//*[@id="spoonacular-ingredient-vis-grid"]/div[{x}]/div/div[4]').text
                    ingredients.append(ingredient)
                    x = x+1
                except(NoSuchElementException):
                    x = "Done"
                    pass
        for element in driver.find_elements(By.ID, "spoonacularPriceBreakdownTable"):
            prices = element.find_element(By.CSS_SELECTOR, "div.spoonacular-quickview").text
            price.append(prices)
        for element in driver.find_elements(By.CSS_SELECTOR, "div.recipe"):
            recipe_name = element.find_element(By.XPATH, '//*[@id="wrapper"]/div/div[3]/h1').text
        for element in driver.find_elements(By.CSS_SELECTOR, "div.spoonacular-nutrient-name"):
            nutrientName.append(element.text)
        for element in driver.find_elements(By.CSS_SELECTOR, "div.spoonacular-nutrient-value"):
            nutrientValue.append(element.text)
        recipe_details = {
            "name": recipe_name,
            "ingredients": dict(zip(ingredients, measures)),
            "price" : price
        }
        try:
            nutrientDictItems = {"Calories" : nutrientValue[nutrientName.index("Calories")],
                    "Protein" : nutrientValue[nutrientName.index("Protein")],
                    "Fat" : nutrientValue[nutrientName.index("Fat")],
                    "Carbs": nutrientValue[nutrientName.index("Carbohydrates")],
                    "Fiber": nutrientValue[nutrientName.index("Fiber")],
                    "Calcium": nutrientValue[nutrientName.index("Calcium")]}
        except (ValueError):
            nutrientDictItems = {"No Information":"Found"}
            
        nutrientDictPer = {recipe_name : nutrientDictItems}
        nutrientDict.update(nutrientDictPer)

        recipe_list.append(recipe_details)
    
    for item in recipe_list:
        choices.append(item["name"])
    return recipe_list

def insert_nutrition():
    clicker = clicked.get()
    pick = nutrientDict[clicker]
    More_Spoonacular_text.delete(1.0, tk.END)

    try:
        
        More_Spoonacular_text.insert(tk.END,f'Calories : {pick["Calories"]}\n')
        More_Spoonacular_text.insert(tk.END,f'Protein : {pick["Protein"]}\n')
        More_Spoonacular_text.insert(tk.END,f'Fat : {pick["Fat"]}\n')
        More_Spoonacular_text.insert(tk.END,f'Carbs : {pick["Carbs"]}\n')
        More_Spoonacular_text.insert(tk.END,f'Fiber : {pick["Fiber"]}\n')
        More_Spoonacular_text.insert(tk.END,f'Calcium : {pick["Calcium"]}\n')
    except KeyError:
        More_Spoonacular_text.insert(tk.END,f'No Nutrition Information Available')
def more_recipe():
    global More_Spoonacular_Label

    
    More_Spoonacular_Frame = Frame(content_frame, background="darkseagreen")
    More_Spoonacular_Frame.grid(row=2, column=0, pady=1)
    More_Spoonacular_Label = Label(More_Spoonacular_Frame, text="Enter The Recipe You want Nutrition\n information for from the dropdown", font=("Arial", 18), background="darkseagreen")
    More_Spoonacular_Label.pack(padx=(50, 50), pady=10)

    
    dropp = OptionMenu(More_Spoonacular_Frame, clicked , *choices) 
    dropp.pack() 



    global More_Spoonacular_text
    More_Spoonacular_text = scrolledtext.ScrolledText(More_Spoonacular_Frame,borderwidth=5, highlightbackground="black", width=60, height=10, wrap=WORD, yscrollcommand=scrollbar.set,font=("Arial", 12))
    More_Spoonacular_text.delete(1.0, tk.END)
    
    fetch_nutrition_button = tk.Button(More_Spoonacular_Frame,borderwidth=5, highlightbackground="black", text="Fetch Nutrition Information", font=("Arial", 18), command=insert_nutrition)
    More_Spoonacular_text.pack(padx=10, pady=10)
    fetch_nutrition_button.pack(padx=10, pady=5)


    
    
#Reset the Recipe_maker Frame
def Recipe_Maker_Reset(recipe_have_foods,conversion_factors,wide_search_input,Single_Nutrition_Input,wide_search_index):
    Ingridient_Nutrition_Frame.grid_forget()
    Recipe_Total_Frame.grid_forget()
    
    Recipe_Maker_Frame.grid(row=1,column=2,pady=0,sticky="ns")
    
    recipe_have_foods = []

    conversion_factors = []


    wide_search_input = []


    Single_Nutrition_Input = []
    wide_search_index = 0
    
  
    
    food_and_id = {}
    return  recipe_have_foods,conversion_factors,wide_search_input,Single_Nutrition_Input,wide_search_index

      


    #Frames
Intro_Frame = Frame(content_frame, background="darkgreen")

Recipe_Frame = Frame(content_frame, background="darkseagreen")

Spacer_Frame = Frame(content_frame,background="darkseagreen")

Recipe_Maker_Frame = Frame(content_frame, background="darkseagreen")
Ingridient_Nutrition_Frame = Frame(content_frame, background="darkseagreen")
Recipe_Total_Frame = Frame(content_frame, background="darkseagreen")

Intro_Label = Label(Intro_Frame, text = "← Find Recipes On The Left", font=("Arial", 17), background="darkgreen",foreground="white")
Intro_Label2 = Label(Intro_Frame, text = "Make Your Own Recipe On The Right →", font=("Arial", 17), background="darkgreen",foreground="white")
Recipe_Label = Label(Recipe_Frame, text = "Enter the Ingredients you want to use \nseparated by a comma (It may \ntake a while to load)", font=("Arial", 18), background="darkseagreen")


fetch_button = tk.Button(Recipe_Frame, text="Fetch Recipes",borderwidth=5, highlightbackground="black", font=("Arial", 18),command = display_recipes)

output_text = scrolledtext.ScrolledText(Recipe_Frame,borderwidth=5, highlightbackground="black", width=60, height=20,wrap=WORD,font=("Arial", 12))

Recipe_Maker_Label = Label(Recipe_Maker_Frame, text = "Select Foods For Your Own Recipe", font=("Arial", 18), background="darkseagreen")
Ingridient_Nutrition_Label = Label(Ingridient_Nutrition_Frame, text = "Single Ingridient Nutrition Stats:", font=("Arial", 18), background="darkseagreen")
option_label = Label(Ingridient_Nutrition_Frame,text="Your Selection",background="darkseagreen",font=("Arial", 13),foreground = "purple2")
nut_text = Text(Ingridient_Nutrition_Frame, font = ("Arial", "12"),height=8, width=20, borderwidth=5, highlightbackground="black")

Recipe_Total_Label = Label(Recipe_Total_Frame, text = "Nutrient Total of Your Recipe:", font=("Arial", 18), background="darkseagreen")
recipeT = Text(Recipe_Total_Frame, font = ("Arial", "12"),height=8, width=20, borderwidth=5, highlightbackground="black")

Recipe_Reset_Button = Button(Recipe_Total_Frame, text ="Reset",
                             command=lambda: Recipe_Maker_Reset(recipe_have_foods,conversion_factors,wide_search_input,Single_Nutrition_Input,wide_search_index))

Intro_Frame.grid(row=0, columnspan=3, pady=1, sticky = "ew") #entire width source: https://intellipaat.com/community/69111/python-tkinter-frame-making-frame-width-span-full-root-window-width

Recipe_Frame.grid(row=1, column=0, pady=1)               


Spacer_Frame.grid(column=1,rowspan=2,sticky="nsew")


Recipe_Maker_Frame.grid(row=1,column=2,pady=0,sticky="ns")


Intro_Label.pack(pady=10)
Intro_Label2.pack(pady=10)

Recipe_Label.pack(padx=(50,50),pady=0)
entry = tk.Entry(Recipe_Frame,borderwidth=5, highlightbackground="black",width=80)
entry.pack(pady=(0, 10))
Recipe_Maker_Label.pack(padx=(50,50),pady=10)
Ingridient_Nutrition_Label.pack(padx=(20,20),pady=10,fill="x")

option_label.pack()
nut_text.pack()
nut_text.configure(state="disabled")

Recipe_Total_Label.pack(padx=(19,20),pady=10,fill="x")
recipeT.pack()
#Recipe_Reset_Button.pack()


output_text.pack(padx=10, pady=10)
fetch_button.pack()

F_Label = Label(Recipe_Maker_Frame, text = "Enter A Food", font=("Arial", 12), background="darkseagreen")
F_Label.pack(pady=10)
F_Entry = Entry(Recipe_Maker_Frame,borderwidth=5, highlightbackground="black",)
F_Entry.pack()
G_Label = Label(Recipe_Maker_Frame, text = "Enter Weight in Grams", font=("Arial", 12), background="darkseagreen")
G_Label.pack(pady=10)
G_Entry = Entry(Recipe_Maker_Frame,borderwidth=5, highlightbackground="black",)
G_Entry.pack()

FG_Button = Button(Recipe_Maker_Frame, text = "Select Food and Gram Weight",borderwidth=5, highlightbackground="black",command=lambda: Recipe_Maker(recipe_have_foods))
FG_Button.pack(pady=10)



Get_Nutrients_Button = Button(Recipe_Maker_Frame,borderwidth=5, highlightbackground="black", text = "Get Nutrients For Your Recipe Below",foreground="purple2",command=lambda: run_conversion_and_wide(recipe_have_foods,wide_search_input, wide_search_index))
Get_Nutrients_Button.pack(pady=5)

Get_Nutrients_Text = Text(Recipe_Maker_Frame, borderwidth=5, highlightbackground="black",width=50, height=2,wrap=WORD, background="darkseagreen",border="0",foreground="purple2")
Get_Nutrients_Text.pack()

root.mainloop()